In [1]:
using ImageView, Images
using Images
using Plots
using DataFrames
using StatsBase
using PyCall
using ScikitLearn, DataFrames
using LinearAlgebra

using BatchIterators
using PyCall
using Images

using PyPlot

skimage = pyimport("skimage")
measure=skimage.measure

job = pyimport("joblib")

PyObject <module 'joblib' from 'C:\\Users\\Dell\\.julia\\conda\\3\\lib\\site-packages\\joblib\\__init__.py'>

In [2]:
struct ECG
    # Define fields if necessary
    
end

function getImage(ecg::ECG, image::AbstractString)
    image = load(image)
    return image
end

function DividingLeads(::ECG, image::Matrix{RGB{N0f8}})
    #dividing the ECG leads from 1-13 from the above image
    Lead_1 = image[300:600, 150:643]
    Lead_2 = image[300:600, 646:1135]
    Lead_3 = image[300:600, 1140:1625]
    Lead_4 = image[300:600, 1630:2125]
    Lead_5 = image[600:900, 150:643]
    Lead_6 = image[600:900, 646:1135]
    Lead_7 = image[600:900, 1140:1625]
    Lead_8 = image[600:900, 1630:2125]
    Lead_9 = image[900:1200, 150:643]
    Lead_10 = image[900:1200, 646:1135]
    Lead_11 = image[900:1200, 1140:1625]
    Lead_12 = image[900:1200, 1630:2125]
    Lead_13 = image[1250:1480, 150:2125]

    Leads=[Lead_1,Lead_2,Lead_3,Lead_4,Lead_5,Lead_6,Lead_7,Lead_8,Lead_9,Lead_10,Lead_11,Lead_12, Lead_13]

    return Leads
end

function PreprocessingLeads(ecg::ECG, DividingLeads::Vector{Matrix{RGB{N0f8}}})
    

    bin_glob=[]
    
    for i in DividingLeads
        hsv_img = HSV.(i)
        channels = channelview(float.(hsv_img))
        hue_img = channels[1,:,:]
        value_img = channels[3,:,:]
        saturation_img = channels[2,:,:]

        mask = zeros(size(hue_img))
        h, s, v = 500, 300, 35
        for ind in eachindex(hue_img)
            if hue_img[ind] <= h && saturation_img[ind] <= s/255 && value_img[ind] <= v/255
                mask[ind] = 1
            end
        end
        binary_img = colorview(Gray, mask)
        binary_img = imresize(binary_img, (300, 450))
        
        push!(bin_glob, binary_img)
        
    end

    
    return bin_glob
    
end


function SignalExtraction_Scaling(ecg::ECG, ecg_preprocessed_leads::Vector{Any})

    
    resized_leads=[]

    for i in ecg_preprocessed_leads
        nonzero_coords = findall(i .!= 0)

        x_coords = [coord[1] for coord in nonzero_coords]
        y_coords = [coord[2] for coord in nonzero_coords]

        combo=imresize(hcat(x_coords, y_coords), (255, 2))
        push!(resized_leads, combo)
    end
    
    return resized_leads


    
end

function CombineConvert1Dsignal(ecg::ECG, ecg_preprocessed_leads::Vector{Any})
    
    scaled_leads=[]


    for i in 1:12

        arr=ecg_preprocessed_leads[i][:,1]

        dt1 = fit(UnitRangeTransform, arr)
        df1=StatsBase.transform(dt1, arr)

        scaled_leads=vcat(scaled_leads, df1)
    end
    
    
    return scaled_leads
    
    
end




function DimensionalReduction(ecg::ECG, ecg_1dsignal::Vector{Any})
    
    result = ScikitLearn.transform(job.load("PCA_ECG (1).pkl"), ecg_1dsignal')
    
    return result
end 
    

function ModelLoad_predict(ecg::ECG, result::Matrix{Float64})
    
    result = job.load("D:\\Cardiovascular-Detection-using-ECG-images-main\\model_test_cv_julia_3.pkl").predict(result)
    return(result)
    
end

ModelLoad_predict (generic function with 1 method)

In [3]:
count_HB=0
count_MI=0
count_PMI=0
count_NO=0
unclassified=0
exceptions=0
count=[]
total=0
#signals_pc_2=[]
#target_2=[]

warn=pyimport("warnings")
warn.filterwarnings("ignore")

time=pyimport("time")


# record start time
start = time.time()



dir_path = "D:\\Cardiovascular-Detection-using-ECG-images-main\\ECG_IMAGES_DATASET"

for file1 in readdir(dir_path)
    #print(file1,"----")
    file_path1 = joinpath(dir_path, file1)

    for file2 in readdir(file_path1)
        # Create an instance of ECG
        ecg=ECG()

        file_path2 = joinpath(file_path1, file2)
        print("\n",file_path2)

        #Call getImage on my_ecg
         image = getImage(ecg, file_path2)
         dividing_leads = DividingLeads(ecg,image)
         ecg_preprocessed_leads = PreprocessingLeads(ecg, dividing_leads)
         ecg_signal_extraction=SignalExtraction_Scaling(ecg, ecg_preprocessed_leads)
         ecg_1dsignal=CombineConvert1Dsignal(ecg, ecg_signal_extraction)
         ecg_final=DimensionalReduction(ecg, ecg_1dsignal)
         result=ModelLoad_predict(ecg,ecg_final)
         #push!(signals_pc_2, ecg_final)
        
        

         if result == [2] && file1=="Abnormal" #0  
             count_HB=count_HB+1
             print("\nECG corresponds to Abnormal ",count_HB)
             


         elseif result == [0] && file1=="MI"  #1  
             count_MI=count_MI+1
             print("\nECG corresponds to Myocardial Infarction(MI) ",count_MI)
             

         elseif result == [1] && file1=="Normal"   #2 
             count_NO=count_NO+1
             print("\nECG corresponds to Normal ",count_NO)
           


         elseif result == [3] && file1=="PMI"  #  
             count_PMI=count_PMI+1
             print("\nECG corresponds to History of Myocardial Infarction(PMI)  ",count_PMI)

         else
             unclassified=unclassified+1
             print("Unclassified ", unclassified)

         end
    end
print("\n")
end

end1 = time.time()


D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(1).jpg
ECG corresponds to Abnormal 1
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(10).jpg
ECG corresponds to Abnormal 2
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(100).jpg
ECG corresponds to Abnormal 3
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(101).jpg
ECG corresponds to Abnormal 4
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(102).jpg
ECG corresponds to Abnormal 5
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(103).jpg
ECG corresponds to Abnormal 6
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(104).jpg
ECG corresponds to Abnormal 7
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(105).jpgUnclassified 1
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAG

D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(161).jpg
ECG corresponds to Abnormal 64
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(162).jpg
ECG corresponds to Abnormal 65
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(163).jpg
ECG corresponds to Abnormal 66
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(164).jpg
ECG corresponds to Abnormal 67
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(165).jpg
ECG corresponds to Abnormal 68
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(166).jpg
ECG corresponds to Abnormal 69
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(167).jpg
ECG corresponds to Abnormal 70
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(168).jpg
ECG corresponds to Abnormal 71
D:\Cardiovascular-Detection-usin

D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(222).jpgUnclassified 13
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(223).jpg
ECG corresponds to Abnormal 126
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(224).jpg
ECG corresponds to Abnormal 127
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(225).jpg
ECG corresponds to Abnormal 128
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(226).jpg
ECG corresponds to Abnormal 129
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(227).jpg
ECG corresponds to Abnormal 130
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(228).jpg
ECG corresponds to Abnormal 131
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(229).jpg
ECG corresponds to Abnormal 132
D:\Cardiovascular-Detection-using-ECG-ima

D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(75).jpg
ECG corresponds to Abnormal 188
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(76).jpg
ECG corresponds to Abnormal 189
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(77).jpg
ECG corresponds to Abnormal 190
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(78).jpg
ECG corresponds to Abnormal 191
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(79).jpg
ECG corresponds to Abnormal 192
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(8).jpg
ECG corresponds to Abnormal 193
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(80).jpg
ECG corresponds to Abnormal 194
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Abnormal\HB(81).jpg
ECG corresponds to Abnormal 195
D:\Cardiovascular-Detection-using

D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(132).jpg
ECG corresponds to Myocardial Infarction(MI) 38
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(133).jpg
ECG corresponds to Myocardial Infarction(MI) 39
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(134).jpg
ECG corresponds to Myocardial Infarction(MI) 40
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(135).jpg
ECG corresponds to Myocardial Infarction(MI) 41
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(136).jpg
ECG corresponds to Myocardial Infarction(MI) 42
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(137).jpg
ECG corresponds to Myocardial Infarction(MI) 43
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(138).jpg
ECG corresponds to Myocardial Infarction(MI) 44
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\MI\MI(1

...

D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(75).jpg
ECG corresponds to Normal 245
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(76).jpg
ECG corresponds to Normal 246
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(77).jpg
ECG corresponds to Normal 247
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(78).jpg
ECG corresponds to Normal 248
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(79).jpg
ECG corresponds to Normal 249
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(8).jpg
ECG corresponds to Normal 250
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(80).jpg
ECG corresponds to Normal 251
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\Normal\Normal(81).jpg
ECG corresponds to Normal 252
D:\Cardiovascular-Detection-using

D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(130).jpgUnclassified 38
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(131).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  33
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(132).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  34
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(133).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  35
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(134).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  36
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(135).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  37
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(136).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  38
D:\Cardiovascular

ECG corresponds to History of Myocardial Infarction(PMI)  79
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(3).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  80
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(30).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  81
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(31).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  82
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(32).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  83
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(33).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  84
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(34).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  85
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DAT

D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(83).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  127
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(84).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  128
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(85).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  129
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(86).jpgUnclassified 63
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(87).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  130
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(88).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  131
D:\Cardiovascular-Detection-using-ECG-images-main\ECG_IMAGES_DATASET\PMI\PMI(89).jpg
ECG corresponds to History of Myocardial Infarction(PMI)  132
D:\Cardiovascular-

1.68070486046769e9

In [4]:
# Calculate the fraction of files in each directory
list = [count_NO / length(readdir("D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Normal")),
        count_HB / length(readdir("D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/Abnormal")),
        count_PMI / length(readdir("D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/PMI")),
        count_MI / length(readdir("D:/Cardiovascular-Detection-using-ECG-images-main/ECG_IMAGES_DATASET/MI")),
        unclassified]

# Create a DataFrame from the list and labels
labels = ["Normal", "Abnormal", "PMI", "MI", "Unclassified"]
l = DataFrame(Label=labels,Values=list)

,Label,Values
,String,Float64
1,Normal,0.950704
2,Abnormal,0.914163
3,PMI,0.825581
4,MI,1.0
5,Unclassified,64.0


In [5]:
print("The time of execution of above program is : ", (end1-start) * 10^3/60000 * 60, " seconds") #

The time of execution of above program is : 167.01844763755798 seconds

In [6]:
print("Overall Accuracy :- ", (count_NO/284+count_HB/233+count_PMI/172+count_MI/239)/4)

Overall Accuracy :- 0.9226121777074263